In [ ]:
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [ ]:
URL = "https://www.putevi-srbije.rs/index.php/en/kategorizacija-vozila-cenovnik-putarine-2"

WAIT_TIME = 5

In [ ]:
# ------------------ DRIVER SETUP ------------------
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument("--headless")  # enable if you want faster runs

driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, WAIT_TIME)

driver.get(URL)

In [ ]:
# ------------------ DATAFRAME SETUP ------------------
columns = [
    "entry_toll_station",
    "exit_toll_station",
    "1aRsd",
    "1Rsd",
    "2Rsd",
    "3Rsd",
    "4Rsd",
    "1aEu",
    "1Eur",
    "2Eur",
    "3Eur",
    "4Eur"
]

rows = []

In [ ]:
# ------------------ HELPERS ------------------
def wait_for_calculation(driver):
    """Wait until calc() updates result fields"""
    WebDriverWait(driver, WAIT_TIME).until(
        lambda d: d.find_element(By.ID, "k1").get_attribute("value") != "0"
    )


def read_results(driver):
    """Read all result inputs in one DOM pass"""
    ids = [
        "k1a", "k1", "k2", "k3", "k4",
        "k1ae", "k1e", "k2e", "k3e", "k4e"
    ]

    data = {}
    for i in ids:
        val = driver.find_element(By.ID, i).get_attribute("value")
        data[i] = float(val.replace(",", ".")) if val else 0.0
    return data

In [ ]:
# ------------------ DROPDOWNS ------------------
entry_select = wait.until(
    EC.presence_of_element_located((By.ID, "ca"))
)
exit_select = wait.until(
    EC.presence_of_element_located((By.ID, "cb"))
)

entry_dropdown = Select(entry_select)
exit_dropdown = Select(exit_select)

entry_options = [
    (opt.get_attribute("value"), opt.text.strip())
    for opt in entry_dropdown.options
]

exit_options = [
    (opt.get_attribute("value"), opt.text.strip())
    for opt in exit_dropdown.options
]

In [ ]:
# ------------------ MAIN LOOP ------------------
for entry_value, entry_name in entry_options:
    if not entry_value.strip():
        continue

    entry_dropdown.select_by_value(entry_value)

    for exit_value, exit_name in exit_options:
        if not exit_value.strip() or exit_value == entry_value:
            continue

        exit_dropdown.select_by_value(exit_value)
        wait_for_calculation(driver)

        results = read_results(driver)

        rows.append({
            "entry_toll_station": entry_name,
            "exit_toll_station": exit_name,
            "1aRsd": results["k1a"],
            "1Rsd":  results["k1"],
            "2Rsd":  results["k2"],
            "3Rsd":  results["k3"],
            "4Rsd":  results["k4"],
            "1aEu":  results["k1ae"],
            "1Eur":  results["k1e"],
            "2Eur":  results["k2e"],
            "3Eur":  results["k3e"],
            "4Eur":  results["k4e"]
        })

        print(f"✔ {entry_name} → {exit_name}")


In [ ]:
# ------------------ DATAFRAME ------------------
df = pd.DataFrame(rows, columns=columns)

df.to_csv("toll_prices.csv", index=False)
print("\n✅ Data saved to toll_prices.csv")

driver.quit()